# Simple EOF analysis based on SVD
A is  3 $\times$ 4 matrix. (location, time) or (n_features, n_samples)

Singular value decomposition
$A\ = U\ \Sigma\ V^T$

$U$ (3 $\times$ 3) = EOF (location, mode)   
$\Sigma$ (3 $\times$ 4)  square roots of the nonzero eigenvalues  
$V$ (4 $\times$ 4)

$ PC = \Sigma\ V^T$ (3 $\times$ 4) (mode, time)

In [37]:
import numpy as np

In [99]:
a = np.array([[2, 4, 1, -6], [8, 25, 1, 2], [5, -3, 4, 1]])
mu = a.mean(axis=0)

# Remove sample mean
a = a - mu

print(a, a.shape)

[[ -3.          -4.66666667  -1.          -5.        ]
 [  3.          16.33333333  -1.           3.        ]
 [  0.         -11.66666667   2.           2.        ]] (3, 4)


In [100]:
eof, sigma, VT = np.linalg.svd(a, full_matrices=True)
smat = np.zeros(a.shape)
smat[:len(sigma), :len(sigma)] = np.diag(sigma)

pc = smat @ VT
pca_score = sigma**2 / (sigma**2).sum()

In [101]:
print('EOF1 =', eof[:, 0])
print('EOF2 =', eof[:, 1])
print('dot product of EOF1 and EOF2 = {:.3f}'.format(eof[:, 0] @ eof[:, 1]))

EOF1 = [ 0.26447791 -0.80122219  0.53674429]
EOF2 = [-0.77247531  0.15719307  0.61528224]
dot product of EOF1 and EOF2 = 0.000


# Reconstruction

We can obtain the original matrix by multiplying eigen vectors and princple components!!

X = EZ

In [102]:
eof @ pc + mu

array([[ 2.,  4.,  1., -6.],
       [ 8., 25.,  1.,  2.],
       [ 5., -3.,  4.,  1.]])

## check $E^T E = I$

In [103]:
I = eof.T @ eof
print(I)
print(np.trace(I), np.allclose(I, np.eye(I.shape[0]), atol=1e-06))

[[ 1.00000000e+00  3.37786034e-18 -2.33237467e-16]
 [ 3.37786034e-18  1.00000000e+00 -5.16664735e-17]
 [-2.33237467e-16 -5.16664735e-17  1.00000000e+00]]
3.000000000000001 True


# Projection

Project arbitary x onto EOF space  
Z = E^T X

In [104]:
#x = np.array([[2, 4], [3, 3], [2, 6]])
x = np.array([[1], [3], [2]])
x.shape

(3, 1)

In [105]:
# projection coefficients
z = eof.T @ x
z

array([[-1.0657001 ],
       [ 0.92966838],
       [ 3.46410162]])

In [106]:
eof @ z

array([[1.],
       [3.],
       [2.]])

### Check if the projection results can be obtained from the least squre fit

In [107]:
z1, res, rank, s = np.linalg.lstsq(eof, x, rcond=None)
z1

array([[-1.0657001 ],
       [ 0.92966838],
       [ 3.46410162]])

### The coefficients do not change even in the truncated EOFs

In [108]:
eof_tr = eof[:, :2]
z1_tr, res, rank, s = np.linalg.lstsq(eof_tr, x, rcond=None)
z1_tr

array([[-1.0657001 ],
       [ 0.92966838]])

In [109]:
eof_tr @ eof_tr.T @ x

array([[-1.00000000e+00],
       [ 1.00000000e+00],
       [-1.55431223e-15]])

In [119]:
EET = eof_tr @ eof_tr.T
EET

array([[ 0.66666667, -0.33333333, -0.33333333],
       [-0.33333333,  0.66666667, -0.33333333],
       [-0.33333333, -0.33333333,  0.66666667]])

In [111]:
np.linalg.inv(eof_tr @ eof_tr.T)

array([[9.00719925e+15, 9.00719925e+15, 9.00719925e+15],
       [9.00719925e+15, 9.00719925e+15, 9.00719925e+15],
       [9.00719925e+15, 9.00719925e+15, 9.00719925e+15]])

In [112]:
x_im = np.linalg.inv(eof_tr @ eof_tr.T)@ x
x_im

array([[5.40431955e+16],
       [5.40431955e+16],
       [5.40431955e+16]])

In [113]:
eof_tr @ eof_tr.T @ x_im

array([[ 5.33333333],
       [-6.66666667],
       [-0.66666667]])

In [114]:
np.linalg.inv(eof_tr @ eof_tr.T) @ eof_tr @ eof_tr.T

array([[ 2.0049636 , -0.53453177, -1.47043183],
       [ 0.52886841,  0.9144099 , -1.44327831],
       [ 0.26447791, -0.80122219,  0.53674429]])

In [115]:
np.linalg.pinv(EET) @ EET

array([[ 0.66666667, -0.33333333, -0.33333333],
       [-0.33333333,  0.66666667, -0.33333333],
       [-0.33333333, -0.33333333,  0.66666667]])

In [116]:
EET @ np.linalg.inv(EET)

array([[ 1.00000000e+00,  3.33333333e-01,  5.00000000e-01],
       [-5.00000000e-01,  3.33333333e-01, -5.00000000e-01],
       [ 2.33146835e-15, -6.66666667e-01,  0.00000000e+00]])

In [117]:
np.linalg.det(EET)

3.7007434154172e-17

In [118]:
eof @ eof.T

array([[ 1.00000000e+00, -9.28077013e-17, -7.56324913e-17],
       [-9.28077013e-17,  1.00000000e+00, -2.80501886e-16],
       [-7.56324913e-17, -2.80501886e-16,  1.00000000e+00]])